In [26]:
#!/usr/bin/env python
# coding: utf-8

# In[29]:

import json
import operator
import string
from nltk import tokenize
from nltk.corpus import stopwords
import pandas as pd
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import untag
from nltk.collocations import *;
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk import ngrams

# works for senti
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.corpus import movie_reviews
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output

# Data is a list of tweets
def getBestDressed(data, kb):
    
    result = []
    
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'golden', 'globe', 'globes']
    stop = remove_terms
    include_terms = ['dress', 'fashion', 'red', 'carpet', 'haute couture', 'gown', 'design', 'look']
    
    for tweet in data:
        
        tweet = re.sub("\d+", "", tweet)       #strip nums
        tweet = re.sub(r'http\S+', '', tweet)  #strip urls
        tweet = re.sub(r'#\S+', '', tweet)     #strip hashtags
        tweet = tweet.translate(translator)    #strip non-alphanumeric characters

        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        
        if any(term in tweet for term in include_terms) and 'suit' not in tweet:
            result.append(tweet)
    
    
    # extract people and put in dictionary with compound scores
    sentiment_analyzer = SentimentIntensityAnalyzer()
    score_dict = {}
    best_dressed_dict = {}
    worst_dressed_dict = {}

    for tweet in result:
        all_scores = sentiment_analyzer.polarity_scores(tweet)
        for k in sorted(all_scores):
            if k == 'compound':
                useful_score = all_scores[k]

        if tweet:
            # Get all possible bigrams & trigrams in a tweet
            gram = list(nltk.everygrams(tweet.split(), 2, 3))

            # Filter through and append to list for tweet
            for g in gram:
                if len(g) == 2:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])):
                        name = ' '.join(g).lower()
                        if useful_score > 0:
                            if name in best_dressed_dict:
                                best_dressed_dict[name] += useful_score
                            else:
                                best_dressed_dict[name] = useful_score
                        if useful_score < 0:
                            if name in worst_dressed_dict:
                                worst_dressed_dict[name] += useful_score
                            else:
                                worst_dressed_dict[name] = useful_score
                else:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[2])):
                        name = ' '.join(g).lower()
                        if useful_score > 0:
                            if name in best_dressed_dict:
                                best_dressed_dict[name] += useful_score
                            else:
                                best_dressed_dict[name] = useful_score
                        if useful_score < 0:
                            if name in worst_dressed_dict:
                                worst_dressed_dict[name] += useful_score
                            else:
                                worst_dressed_dict[name] = useful_score
        
        
    # look in kb for matches     
    final_dict = {}
    for best, worst in zip(best_dressed_dict, worst_dressed_dict):
        if best.lower() in kb:
            final_dict[best] = best_dressed_dict[best]
        if worst.lower() in kb:
            final_dict[worst] = worst_dressed_dict[worst]
            

    # get key with max value
    best_dressed = []
    worst_dressed = []
    
    while len(best_dressed) < 6:
        best_person = max(final_dict.items(), key=lambda k: k[1])[0]
        worst_person = min(final_dict.items(), key=lambda k: k[1])[0]
        best_dressed.append(best_person)
        worst_dressed.append(worst_person)
        del final_dict[best_person]
        del final_dict[worst_person]
    
    return best_dressed, worst_dressed

/Users/ikhlas/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [31]:
with open('../people_kb.json') as json_file:
    kb = json.load(json_file)


data = pd.read_json('../gg2013.json')
data = pd.DataFrame(data)
data = data['text'].values.tolist()

["JLo's dress! #eredcarpet #GoldenGlobes",
 "What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes",
 'RT @FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! @goldenglobes #goldenglobes http://t.co/3feH7MvO',
 'Anne Hathaway has got me living. #GoldenGlobes',
 "Jennifer Lopez's lace dress? Thoughts? #GoldenGlobes",
 'Podrán criticar a #Adele de su moda y su maniquere pero jamás podrán poner en tela de juicio su TALENTO! @eonline #GoldenGlobes #RedCarpet',
 'US, Weakly #GoldenGlobes',
 'RT @BillMc7: "Wait. What\'s that smell?!" (everyone being interviewed by Al Roker) #GoldenGlobes',
 'Hugh Jackman is so awesome!!! #goldenglobes',
 'It was awkward. RT @hollywoodhwife: They cut to and then from JLo during the Ben Affleck interview.  #goldenglobes',
 'hellen mirren *O* #goldenglobes',
 "Jennifer Lopez's dress is jaw droppingly amazing #GoldenGlobes #redcarpet",
 'Jessica Alba is absolutely flawless, omg❤ #redcarpet #GoldenGlobes',
 'Werk 

In [28]:
def getCouple(data, kb):
    
    result = []
    
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'golden', 'globe', 'globes']
    stop = remove_terms
#     include_terms = ['dress', 'fashion', 'red', 'carpet', 'haute couture', 'gown', 'design', 'look']
    
    for tweet in data:
        
        tweet = re.sub("\d+", "", tweet)       #strip nums
        tweet = re.sub(r'http\S+', '', tweet)  #strip urls
        tweet = re.sub(r'#\S+', '', tweet)     #strip hashtags
        tweet = tweet.translate(translator)    #strip non-alphanumeric characters

        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        
#         if any(term in tweet for term in include_terms) and 'suit' not in tweet:
#             result.append(tweet)
    
    
    # extract people and put in dictionary with compound scores
    sentiment_analyzer = SentimentIntensityAnalyzer()
    score_dict = {}
    best_couple_dict = {}
    worst_couple_dict = {}

    for tweet in result:
        all_scores = sentiment_analyzer.polarity_scores(tweet)
        for k in sorted(all_scores):
            if k == 'compound':
                useful_score = all_scores[k]

        if tweet:
            # Get all possible bigrams & trigrams in a tweet
            gram = list(nltk.everygrams(tweet.split(), 2, 3))

            # Filter through and append to list for tweet
            for g,n in gram:
                if len(g) == 4:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', n[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', n[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) != bool(re.match(r'\b[A-Z][a-z]+\b', n[0])):
                        name1 = ' '.join(g).lower()
                        name2 = ' '.join(n).lower()
                        if useful_score > 0 and name1 != name2:
                            if name1+' and '+name2 in best_couple_dict:
                                best_couple_dict[name1+' and '+name2] += useful_score
                            else:
                                best_couple_dict[name1+' and '+name2] = useful_score
                        elif useful_score < 0:
                            if name in worst_couple_dict:
                                worst_couple_dict[name1+' and '+name2] += useful_score
                            else:
                                worst_couple_dict[name1+' and '+name2] = useful_score
                                
            # check amount og capitalized grams
                else:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[2])) and bool(re.match(r'\b[A-Z][a-z]+\b', n[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', n[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', n[2])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) != bool(re.match(r'\b[A-Z][a-z]+\b', n[0])):
                        name1 = ' '.join(g).lower()
                        name2 = ' '.join(n).lower()
                        if useful_score > 0 and name1 != name2:
                            if name1+' and '+name2 in best_couple_dict:
                                best_couple_dict[name1+' and '+name2] += useful_score
                            else:
                                best_couple_dict[name1+' and '+name2] = useful_score
                        elif useful_score < 0:
                            if name in worst_couple_dict:
                                worst_couple_dict[name1+' and '+name2] += useful_score
                            else:
                                worst_couple_dict[name1+' and '+name2] = useful_score
        
    # look in kb for matches     
    final_dict_c = {}
    for best, worst in zip(best_couple_dict, worst_couple_dict):
        if best.lower() in kb:
            final_dict_c[best] = best_couple_dict[best]
        if worst.lower() in kb:
            final_dict_c[worst] = worst_couple_dict[worst]
    
    # get key with max value
    best_couple = []
    worst_couple = []
    
    while len(best_couple) < 12:
        best_persons = max(final_dict_c.items(), key=lambda k: k[1])[0]
        worst_persons = min(final_dict_c.items(), key=lambda k: k[1])[0]
        best_couple.append(best_persons)
        worst_couple.append(worst_persons)
        del final_dict_c[best_persons]
        del final_dict_c[worst_persons]
    
    return best_couple, worst_couple

In [30]:
getCouple(data,kb)

ValueError: max() arg is an empty sequence

In [ ]:
def getJoke(data):
    
    result = []
    
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'golden', 'globe', 'globes']
    stop = remove_terms
    include_terms = ['joke','laugh','haha','funny','hilarious','lol','lmao']
    
    for tweet in data:
        
        tweet = re.sub("\d+", "", tweet)       #strip nums
        tweet = re.sub(r'http\S+', '', tweet)  #strip urls
        tweet = re.sub(r'#\S+', '', tweet)     #strip hashtags
        tweet = tweet.translate(translator)    #strip non-alphanumeric characters

        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        
        if any(term in tweet for term in include_terms) not in tweet:
            result.append(tweet)
    
    # extract people and put in dictionary with compound scores
    sentiment_analyzer = SentimentIntensityAnalyzer()
    score_dict = {}
    best_joke_dict = {}

    for tweet in result:
        all_scores = sentiment_analyzer.polarity_scores(tweet)
        for k in sorted(all_scores):
            if k == 'compound':
                useful_score = all_scores[k]

        if tweet:
            # Get all possible bigrams & trigrams in a tweet
            gram = list(nltk.everygrams(tweet.split(), 2, 3))

            # Filter through and append to list for tweet
            for g in gram:
                if len(g) == 2:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])):
                        name = ' '.join(g).lower()
                        if useful_score > 0:
                            if name in best_dressed_dict:
                                best_dressed_dict[name] += useful_score
                            else:
                                best_dressed_dict[name] = useful_score
                        if useful_score < 0:
                            if name in worst_dressed_dict:
                                worst_dressed_dict[name] += useful_score
                            else:
                                worst_dressed_dict[name] = useful_score
                else:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[2])):
                        name = ' '.join(g).lower()
                        if useful_score > 0:
                            if name in best_dressed_dict:
                                best_dressed_dict[name] += useful_score
                            else:
                                best_dressed_dict[name] = useful_score
                        if useful_score < 0:
                            if name in worst_dressed_dict:
                                worst_dressed_dict[name] += useful_score
                            else:
                                worst_dressed_dict[name] = useful_score
        
        
    # look in kb for matches     
    final_dict = {}
    for best, worst in zip(best_dressed_dict, worst_dressed_dict):
        if best.lower() in kb:
            final_dict[best] = best_dressed_dict[best]
        if worst.lower() in kb:
            final_dict[worst] = worst_dressed_dict[worst]
            

    # get key with max value
    best_dressed = []
    worst_dressed = []
    
    while len(best_dressed) < 6:
        best_person = max(final_dict.items(), key=lambda k: k[1])[0]
        worst_person = min(final_dict.items(), key=lambda k: k[1])[0]
        best_dressed.append(best_person)
        worst_dressed.append(worst_person)
        del final_dict[best_person]
        del final_dict[worst_person]
    
    return best_dressed, worst_dressed